Lloyd algorithm in m dimensions

In [237]:
import pandas as pd
import numpy as np
import math

data = pd.read_table('/Users/tim/Desktop/Data Mining/HW 5/rosalind_ba8c.txt', delimiter = " ")
k,m = int(data.columns[0]), int(data.columns[1])
data = pd.read_table('/Users/tim/Desktop/Data Mining/HW 5/rosalind_ba8c.txt', delimiter = " ", skiprows = 1, header = None)
centers = data.iloc[0:k,:]

def EuclideanDistance(point, center):
    distances = [(point[s] - center[s])**2 for s in range(0,m)]
    distance = math.sqrt(sum(distances))
    return distance

def closestcenter(point, centers):
    minDistance = float("Inf")
    closest = centers.iloc[0,:]
    for t in range(0,len(centers)):
        dist = EuclideanDistance(point,centers.iloc[t,:])
        if dist < minDistance:
            minDistance = dist
            closest = centers.iloc[t,:]
    return closest
    
def mean_cluster(clusterp):
    center = [0]*m
    for point in clusterp:
        for i in range(0,m):
            center[i] += point[i]        
    return np.divide(center,len(clusterp))

def lloyd(data, centers):
    while math.factorial(0)==1:
        
        # from points to clusters
        cluster_dict = {}
        for ind in range(0,len(data)):
            point = data.iloc[ind,:]
            ccenter = tuple(closestcenter(point, centers))
            if ccenter in cluster_dict:
                # we could have also used defaultdict initially
                if not isinstance(cluster_dict[ccenter], list):
                    cluster_dict[ccenter] = [cluster_dict[ccenter]]
                cluster_dict[ccenter].append(tuple(point))
            else:
                cluster_dict[ccenter] = tuple(point)
        # from clusters to points
        ncenters = pd.DataFrame()
        for cluster in cluster_dict:
            ncenters = pd.concat([ncenters, pd.DataFrame(mean_cluster(cluster_dict[tuple(cluster)]))], ignore_index=True, axis=1)
        # convergence check
        ncenters = ncenters.T
        if centers.equals(ncenters):
            break
        centers = ncenters
    return centers

centers = lloyd(data,centers)

centers = centers.round(decimals=3)
for i in range(0,len(centers)):    
    print(" ".join(map(str, centers.iloc[i,:])))  

18.289 6.607 7.977 7.057
7.363 7.238 7.894 20.872
5.985 5.583 4.902 3.37
4.231 5.44 4.367 11.444
6.478 18.019 6.118 6.319
5.707 6.596 16.904 6.701


Visualization in 2D (up to five centroids)

In [241]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
import os

datax = np.random.random(250)
datay = np.random.random(250)
data = np.vstack((datax,datay)).T; k,m = 4,2
centers = data[0:k][:]

def EuclideanDistance(point, center):
    distances = [(point[s] - center[s])**2 for s in range(0,m)]
    distance = math.sqrt(sum(distances))
    return distance

def closestcenter(point, centers):
    minDistance = float("Inf")
    closest = centers[0][:]
    for t in range(0,len(centers)):
        dist = EuclideanDistance(point,centers[t][:])
        if dist < minDistance:
            minDistance = dist
            closest = centers[t][:]
    return closest
    
def mean_cluster(clusterp):
    center = [0]*m
    #clusterp = [clusterp]
    for point in clusterp:
        for i in range(0,m):
            center[i] += point[i]        
    return np.divide(center,len(clusterp))

def lloyd(data, centers):
    frame = 0
    while math.factorial(0)==1:
        for point in centers:
            plt.scatter(point[0],point[1],color = 'grey',marker = '*',s = 150)
        plt.ylim(0, 1)
        plt.xlim(0, 1)
        plt.title('Centroids after %d iteration(s)' %(frame/2+1))
        filename = f'{frame}.png'
        filenames.append(filename)
        # save frame
        plt.savefig(filename, dpi = 360)
        plt.close()
        frame += 1
        # from points to clusters
        cluster_dict = {}
        cluster_dict_col = {}
        colors = ['blue','green','orange','pink','black']
        colorometer = 0
        for ind in range(0,len(data)):
            point = data[ind][:]
            ccenter = tuple(closestcenter(point, centers))
            # dictionary for colors
            if ccenter not in cluster_dict_col:
                cluster_dict_col[ccenter] = colors[colorometer]
                colorometer += 1
            # dictionary for closest centers
            if ccenter in cluster_dict:
                # we could have also used defaultdict initially
                if not isinstance(cluster_dict[ccenter], list):
                    cluster_dict[ccenter] = [cluster_dict[ccenter]]
                cluster_dict[ccenter].append(tuple(point))
                plt.scatter(point[0],point[1],color = cluster_dict_col[ccenter])
            else:
                cluster_dict[ccenter] = tuple(point)
                plt.scatter(ccenter[:][0],ccenter[:][1],color = cluster_dict_col[ccenter],marker = '*',s = 150)
        # from clusters to points
        plt.title('Clustering after %d iteration(s)' %(frame/2+1))
        filename = f'{frame}.png'
        filenames.append(filename)
        # save frame
        plt.savefig(filename, dpi = 360)
        plt.close()
        frame += 1
        ncenters = np.empty((0,m))
        for cluster in cluster_dict:
            ncenters = np.vstack([ncenters, mean_cluster(cluster_dict[tuple(cluster)])])
        # convergence check
        if np.array_equal(centers,ncenters):
            break
        centers = ncenters  
    return centers

filenames = []
centers = lloyd(data,centers)

centers = centers.round(decimals=3)
for i in range(0,len(centers)):    
    print(" ".join(map(str, centers[i][:])))  
    
# Build GIF, credit: https://towardsdatascience.com/basics-of-gifs-with-pythons-matplotlib-54dd544b6f30
with imageio.get_writer('Lloyd.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
# Remove files
for filename in set(filenames):
    os.remove(filename)

0.247 0.287
0.769 0.748
0.748 0.242
0.253 0.765
